# How much gets filtered when removing rRNA, tRNA, and mtDNA/RNA? 
I'm making this notebook as a n intermission from preparing rolypoly external database. So far I've used a combination of NCBI +SILVA rRNAs to remove contaminating rRNA reads from RNA-seq data. Subsequently, the organisms whose rRNAs were most matched, are then also fetched (or their transcriptomes, if available) to remove any remaining reads that may have come from those organisms.  
THis is messy, requires NCBI taxdump, taxonkit, and ncbi-datasets. So I started removing this dependency by using a set of rRNAs for which I can generate a prebuilt table containing the FTP addresses of the hosts' genomes/transcriptomes.  
While doing so, I realised the step above could be split - quick rRNA mapping to get rough taxonomic breakdown of the sample, and then a more thorough removal of rRNA, tRNA, and mtDNA/RNA using a more comprehensive database. The question is then how much will these diffrent combinations filter, how much more time, and would masking the fasta for subsequences shared with RNA viruses change the results significantly?

To test these, below are how I got the data, how I named the sets, filtering of them, graphs and so on. 

*Note*: Parts of this script are from [build_data.py](../src/rolypoly/commands/misc/build_data.py) and [filter_reads.py](../src/rolypoly/commands/reads/filter_reads.py) scripts in the rolypoly repository.

Loading libraries and defining paths to sets already created/downloaded:

In [53]:
import datetime
import json
import logging
import os
import shutil
import subprocess
import tarfile
from pathlib import Path as pt

import polars as pl
import requests
from bbmapy import bbduk, bbmask, kcompress
from rich.console import Console
from rich_click import command, option

from rolypoly.utils.bio.alignments import (
    hmmdb_from_directory,
    mmseqs_profile_db_from_directory,
)
from rolypoly.utils.bio.sequences import (
    filter_fasta_by_headers,
    remove_duplicates,
)

from rolypoly.utils.bio.polars_fastx import from_fastx_eager

from rolypoly.utils.logging.citation_reminder import remind_citations
from rolypoly.utils.logging.loggit import get_version_info, setup_logging
from rolypoly.utils.various import fetch_and_extract, run_command_comp

### DEBUG ARGS (for manually building, not entering via CLI):
threads = 6
log_file = "notebooks/Exprimental/trrna.log"
data_dir = "/clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data"

global rrna_dir
global contam_dir

logger = setup_logging(log_file)
print(f"Starting data preparation to : {data_dir}")

contam_dir = os.path.join(data_dir, "contam")
os.makedirs(contam_dir, exist_ok=True)

rrna_dir = os.path.join(contam_dir, "rrna")
os.makedirs(rrna_dir, exist_ok=True)

masking_dir = os.path.join(contam_dir, "masking")
os.makedirs(masking_dir, exist_ok=True)

# taxonomy_dir = os.path.join(data_dir, "taxdump")
# os.makedirs(taxonomy_dir, exist_ok=True)

reference_seqs = os.path.join(data_dir, "reference_seqs")
os.makedirs(reference_seqs, exist_ok=True)

mmseqs_ref_dir = os.path.join(reference_seqs, "mmseqs")
os.makedirs(mmseqs_ref_dir, exist_ok=True)

rvmt_dir = os.path.join(reference_seqs, "RVMT")
os.makedirs(rvmt_dir, exist_ok=True)

ncbi_ribovirus_dir = os.path.join(reference_seqs, "ncbi_ribovirus")
os.makedirs(ncbi_ribovirus_dir, exist_ok=True)

# Masking sequences preparation
rvmt_fasta_path = os.path.join(
    data_dir, "reference_seqs", "RVMT", "RVMT_cleaned_contigs.fasta"
)
ncbi_ribovirus_fasta_path = os.path.join(
    data_dir,
    "reference_seqs",
    "ncbi_ribovirus",
    "refseq_ribovirus_genomes.fasta",
)

rna_viruses_entropy_masked_path = os.path.join(
    masking_dir, "combined_entropy_masked.fasta"
)

Starting data preparation to : /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data


## Fetching data (SILVA, NCBI rRNAs, tRNAs, mtDNAs):

In [54]:
silva_release = "138.2"

# Download SILVA rRNA sequences (SSU and LSU)
silva_ssu_path = os.path.join(
    rrna_dir, f"SILVA_{silva_release}_SSURef_NR99_tax_silva.fasta"
)
silva_lsu_path = os.path.join(
    rrna_dir, f"SILVA_{silva_release}_LSURef_NR99_tax_silva.fasta"
)
# skipping downloading (already done)
# fetch_and_extract(
#     f"https://www.arb-silva.de/fileadmin/silva_databases/release_{silva_release.replace('.', '_')}/Exports/SILVA_{silva_release}_SSURef_NR99_tax_silva.fasta.gz",
#     fetched_to=os.path.join(rrna_dir, "tmp_ssu.fasta.gz"),
#     extract_to=rrna_dir,
#     rename_extracted=silva_ssu_path,
#     logger=logger,
# )
# fetch_and_extract(
#     f"https://www.arb-silva.de/fileadmin/silva_databases/release_{silva_release.replace('.', '_')}/Exports/SILVA_{silva_release}_LSURef_NR99_tax_silva.fasta.gz",
#     fetched_to=os.path.join(rrna_dir, "tmp_lsu.fasta.gz"),
#     extract_to=rrna_dir,
#     rename_extracted=silva_lsu_path,
#     logger=logger,
# )

# Download SILVA taxonomy mappings (maps accessions to NCBI taxids)
silva_ssu_taxmap = pl.read_csv(
    "https://www.arb-silva.de/fileadmin/silva_databases/current/Exports/taxonomy/ncbi/taxmap_embl-ebi_ena_ssu_ref_nr99_138.2.txt.gz",
    truncate_ragged_lines=True,
    separator="\t",
    infer_schema_length=123123,
)
silva_lsu_taxmap = pl.read_csv(
    "https://www.arb-silva.de/fileadmin/silva_databases/current/Exports/taxonomy/ncbi/taxmap_embl-ebi_ena_lsu_ref_nr99_138.2.txt.gz",
    truncate_ragged_lines=True,
    separator="\t",
    infer_schema_length=123123,
)
silva_taxmap = pl.concat([silva_lsu_taxmap, silva_ssu_taxmap])
silva_taxmap.write_parquet(os.path.join(rrna_dir, "silva_taxmap_embl-ebi_ena.parquet"))

# Parse SILVA headers and extract accessions
silva_fasta_df = pl.concat(
    [
        from_fastx_eager(silva_ssu_path).with_columns(
            pl.lit("SSU").alias("rRNA_type")
        ),
        from_fastx_eager(silva_lsu_path).with_columns(
            pl.lit("LSU").alias("rRNA_type")
        ),
    ]
)
silva_fasta_df.write_parquet(os.path.join(rrna_dir, "silva99_fasta.parquet"))
print(f"total SILVA99 sequences {silva_fasta_df.height}")
silva_fasta_df.head(4)


total SILVA99 sequences 605774


header,sequence,rRNA_type
str,str,str
"""AY846379.1.1791 Eukaryota;Arch…","""AACCUGGUUGAUCCUGCCAGUAGUCAUAUG…","""SSU"""
"""AY846382.1.1778 Eukaryota;Arch…","""GUUGAUCCUGCCAGUAGUCAUAUGCUUGUC…","""SSU"""
"""AB000393.1.1510 Bacteria;Pseud…","""UGGCUCAGAUUGAACGCUGGCGGCAGGCCU…","""SSU"""
"""AY909590.1.2352 Eukaryota;Arch…","""GACUAAGCCAUGCAUGUCUAAGUAUAAACG…","""SSU"""


In [55]:
# Extract accession from header (format: >accession.version rest_of_header)
silva_fasta_df = silva_fasta_df.with_columns(
    primaryAccession=pl.col("header").str.extract(
        r"^([A-Za-z0-9_]+)(?:\.\d+)*", 1
    ),  # DQ150555.1.2478 -> DQ150555
    accession=pl.col("header").str.extract(
        r"^([A-Za-z0-9_]+(?:\.\d+)?)", 1
    ),  # AY846379 or DQ150555.1
    taxonomy_raw=pl.col("header").str.replace(r"^\S+\s+", ""),
)
    # silva_fasta_df = silva_fasta_df.with_columns(
    #     pl.col("sequence").str.len_chars().alias("seq_length")
    # )
    # silva_taxmap = silva_taxmap.with_columns(
    #     (pl.col("stop") - pl.col("start")).alias("seq_length")
    # )

silva_df = silva_fasta_df.join(
    silva_taxmap.select(
        ["primaryAccession", "ncbi_taxonid", "submitted_path"]
    ).unique(),  # seq_length
    on=["primaryAccession"],
    how="inner",
)
silva_df.write_parquet(os.path.join(rrna_dir, "silva_rrna_sequences.parquet"))
# silva_df.height
silva_df["ncbi_taxonid"].null_count()

# Load SILVA taxonomy mappings
print(
    f"Merged taxonomy for {silva_df.filter(pl.col('ncbi_taxonid').is_not_null()).height} SILVA sequences"
)

unique_taxids = (
    silva_df.filter(pl.col("ncbi_taxonid").is_not_null())
    .select("ncbi_taxonid")
    .unique()["ncbi_taxonid"]
    .to_list()
)
print(
    f"Total of {len(unique_taxids)} unique NCBI taxids found in SILVA sequences"
)


Merged taxonomy for 605774 SILVA sequences
Total of 105645 unique NCBI taxids found in SILVA sequences


In [3]:
# silva_df = pl.read_parquet(os.path.join(rrna_dir, "silva_rrna_sequences.parquet"))

In [ ]:
# Generate FTP download URLs for host genomes/transcriptomes
fetch_and_extract(
    url="https://ftp.ncbi.nlm.nih.gov/genomes/genbank/assembly_summary_genbank.txt",
    fetched_to=os.path.join(rrna_dir, "assembly_summary_genbank.txt.gz"),
    extract=False,
)
print("Loading NCBI GenBank assembly summary")
# genbank_summary = pl.read_csv(os.path.join(rrna_dir, "assembly_summary_genbank.txt.gz",),
# infer_schema_length=100020, separator="\t", skip_rows=1,
# null_values=["na","NA","-"],ignore_errors=True,
# has_header=True)
# polars failed me, so using line by line iterator
from gzip import open as gz_open
with gz_open(
    os.path.join(rrna_dir, "assembly_summary_genbank.txt.gz"), "r"
) as f:
    header = None
    records = []
    i = 0
    for line in f:
        if i == 0:
            i += 1
            continue
        line = line.rstrip(b"\n")
        if i == 1:
            header = line.decode()[1:].strip().split("\t")
            i += 1
            continue
        fields = line.decode().strip().split("\t")
        record = dict(zip(header, fields))
        records.append(record)
genbank_summary = pl.from_records(records).rename({"taxid": "ncbi_taxonid"})
genbank_summary.collect_schema()
    # Schema([('assembly_accession', String),
#         ('bioproject', String),
#         ('biosample', String),
#         ('wgs_master', String),
#         ('refseq_category', String),
#         ('ncbi_taxonid', String),
#         ('species_taxid', String),
#         ('organism_name', String),
#         ('infraspecific_name', String),
#         ('isolate', String),
#         ('version_status', String),
#         ('assembly_level', String),
#         ('release_type', String),
#         ('genome_rep', String),
#         ('seq_rel_date', String),
#         ('asm_name', String),
#         ('asm_submitter', String),
#         ('gbrs_paired_asm', String),
#         ('paired_asm_comp', String),
#         ('ftp_path', String),
#         ('excluded_from_refseq', String),
#         ('relation_to_type_material', String),
#         ('asm_not_live_date', String),
#         ('assembly_type', String),
#         ('group', String),
#         ('genome_size', String),
#         ('genome_size_ungapped', String),
#         ('gc_percent', String),
#         ('replicon_count', String),
#         ('scaffold_count', String),
#         ('contig_count', String),
#         ('annotation_provider', String),
#         ('annotation_name', String),
#         ('annotation_date', String),
#         ('total_gene_count', String),
#         ('protein_coding_gene_count', String),
#         ('non_coding_gene_count', String),
#         ('pubmed_id', String)])

genbank_summary.write_csv(
    os.path.join(rrna_dir, "genbank_assembly_summary.tsv"), separator="\t"
)
genbank_summary = pl.read_csv(
    os.path.join(rrna_dir, "genbank_assembly_summary.tsv"),
    infer_schema_length=100020,
    separator="\t",
    null_values=["na", "NA", "-"],
    ignore_errors=True,
    has_header=True,
)

# In [91]: genbank_summary.collect_schema()
# Out[91]: 
# Schema([('assembly_accession', String),
#         ('bioproject', String),
#         ('biosample', String),
#         ('wgs_master', String),
#         ('refseq_category', String),
#         ('ncbi_taxonid', Int64),
#         ('species_taxid', Int64),
#         ('organism_name', String),
#         ('infraspecific_name', String),
#         ('isolate', String),
#         ('version_status', String),
#         ('assembly_level', String),
#         ('release_type', String),
#         ('genome_rep', String),
#         ('seq_rel_date', String),
#         ('asm_name', String),
#         ('asm_submitter', String),
#         ('gbrs_paired_asm', String),
#         ('paired_asm_comp', String),
#         ('ftp_path', String),
#         ('excluded_from_refseq', String),
#         ('relation_to_type_material', String),
#         ('asm_not_live_date', String),
#         ('assembly_type', String),
#         ('group', String),
#         ('genome_size', Int64),
#         ('genome_size_ungapped', Int64),
#         ('gc_percent', Float64),
#         ('replicon_count', Int64),
#         ('scaffold_count', Int64),
#         ('contig_count', Int64),
#         ('annotation_provider', String),
#         ('annotation_name', String),
#         ('annotation_date', String),
#         ('total_gene_count', Int64),
#         ('protein_coding_gene_count', Int64),
#         ('non_coding_gene_count', Int64),
#         ('pubmed_id', String)])

genbank_summary.write_parquet(
    os.path.join(rrna_dir, "genbank_assembly_summary.parquet")
)
genbank_summary.write_csv(
    os.path.join(rrna_dir, "genbank_assembly_summary.tsv"), separator="\t"
)
print(genbank_summary.head(4))

2025-12-01 14:15:15,584 - rolypoly.utils.logging.loggit - INFO - Downloading https://ftp.ncbi.nlm.nih.gov/genomes/genbank/assembly_summary_genbank.txt to /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/assembly_summary_genbank.txt.gz
2025-12-01 14:15:45,465 - rolypoly.utils.logging.loggit - INFO - Successfully downloaded to /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/assembly_summary_genbank.txt.gz


INFO     2025-12-01 14:15:45 - Loading NCBI GenBank assembly summary                                                                   ]8;id=290238;file:///tmp/ipykernel_2638087/3124954572.py\3124954572.py]8;;\:]8;id=190348;file:///tmp/ipykernel_2638087/3124954572.py#7\7]8;;\

In [56]:
genbank_summary = pl.read_parquet(
    os.path.join(rrna_dir, "genbank_assembly_summary.parquet")
)
print(genbank_summary.head(4))

shape: (4, 38)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ assembly_ ┆ bioprojec ┆ biosample ┆ wgs_maste ┆ … ┆ total_gen ┆ protein_c ┆ non_codin ┆ pubmed_i │
│ accession ┆ t         ┆ ---       ┆ r         ┆   ┆ e_count   ┆ oding_gen ┆ g_gene_co ┆ d        │
│ ---       ┆ ---       ┆ str       ┆ ---       ┆   ┆ ---       ┆ e_count   ┆ unt       ┆ ---      │
│ str       ┆ str       ┆           ┆ str       ┆   ┆ i64       ┆ ---       ┆ ---       ┆ str      │
│           ┆           ┆           ┆           ┆   ┆           ┆ i64       ┆ i64       ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ GCA_00000 ┆ PRJNA1381 ┆ SAMN02803 ┆ null      ┆ … ┆ 17872     ┆ 13962     ┆ 3543      ┆ 10731132 │
│ 1215.4    ┆ 2         ┆ 731       ┆           ┆   ┆           ┆           ┆           ┆ ;1253756 │
│           ┆           ┆           ┆           ┆   ┆           ┆           

In [57]:
# next, for every unique ncbi_taxonid, we select the one that has the most protein_coding_gene_count, then refseq_category, then tie breaking with non_coding_gene_count, tie breaking by latest assembly (by seq_rel_date).
mini_genebank = genbank_summary.sort(
    by=[
        pl.col("refseq_category").reverse(),
        pl.col("protein_coding_gene_count").cast(pl.Int64).reverse(),
        pl.col("non_coding_gene_count").cast(pl.Int64).reverse(),
        pl.col("seq_rel_date").reverse(),
        pl.col("genome_size").reverse(),
    ]
).unique(subset=["ncbi_taxonid"], keep="first")
print(
    f"Filtered GenBank summary to {mini_genebank.height} unique taxid entries for SILVA sequences"
)

in_silva = mini_genebank.filter(pl.col("ncbi_taxonid").is_in(silva_taxmap["ncbi_taxonid"])).unique()
print(in_silva.height)
# print(temp_genbank.height)
# only 30503 out of ~240k?


Filtered GenBank summary to 242839 unique taxid entries for SILVA sequences
30503


/tmp/ipykernel_1531896/1389620562.py:15: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  in_silva = mini_genebank.filter(pl.col("ncbi_taxonid").is_in(silva_taxmap["ncbi_taxonid"])).unique()


In [58]:
# next, for every unique entry, we create paths for the expected location of the transcripts/transcriptome fasta, the rna fasta, and the genome.
mini_genebank = mini_genebank.with_columns(
        (pl.col("ftp_path") + "/md5checksums.txt").alias("md5sums_path")
)
# fetching the md5sums files for a subset to test time
test_df = mini_genebank[:10].with_columns(
        pl.col("ftp_path").map_elements(
            lambda path: requests.get(path + "/md5checksums.txt").text,return_dtype=pl.Utf8
        ).alias("md5sums_content")
)
test_df["md5sums_content"].first()


'ba7bb432c803cec66a415cf43ae54ef6  ./annotation_hashes.txt\nd364bab756fa59130f83f23bface8fc1  ./GCA_018101015.1_ASM1810101v1_assembly_report.txt\nf501bb76bb9b9d7bf21d5acf8c373195  ./GCA_018101015.1_ASM1810101v1_assembly_stats.txt\n1aeebbbf8a82ce15c162d3cc43012076  ./GCA_018101015.1_ASM1810101v1_cds_from_genomic.fna.gz\n3eb62af3551174adf510b43c50973ae4  ./GCA_018101015.1_ASM1810101v1_feature_count.txt.gz\n7bd5fb2dce5f65772654387c0ec54494  ./GCA_018101015.1_ASM1810101v1_feature_table.txt.gz\n8ab57f1768639bb819782b1a9662fc5d  ./GCA_018101015.1_ASM1810101v1_genomic.fna.gz\n8f6bd8a2f355588abc10f4e961f4a9fc  ./GCA_018101015.1_ASM1810101v1_genomic.gbff.gz\n74b9e1552fa5a08548e4c65a112b9251  ./GCA_018101015.1_ASM1810101v1_genomic.gff.gz\n3784cb6b6ed9b2ee39bf341c8503aa72  ./GCA_018101015.1_ASM1810101v1_genomic.gtf.gz\n4971e393f496187e20fe39322a40f4ac  ./GCA_018101015.1_ASM1810101v1_protein.faa.gz\nbb8fadb4e8c6ab69688aa01f8a2fad5c  ./GCA_018101015.1_ASM1810101v1_protein.gpff.gz\nfc0f673ef99774562

In [94]:
# 2.2s, too slow, so will do it with aria2 using proper FTP protocol
import aria2p
import tempfile
import os
import time
from tqdm.notebook import tqdm

# Create a temporary directory for downloads
temp_dir = tempfile.mkdtemp()

# Start aria2c daemon (assumes aria2c is running with RPC enabled)
api = aria2p.API(
    aria2p.Client(
        host="http://localhost",
        port=6800,
        secret=""
    )
)
# print("Make sure aria2c is running with: aria2c --enable-rpc --rpc-listen-all=true --rpc-allow-origin-all")

def convert_to_ftp_url(http_ftp_path):
    """Convert NCBI HTTP FTP path to proper FTP URL"""
    if http_ftp_path.startswith("https://ftp.ncbi.nlm.nih.gov/"):
        # Convert https://ftp.ncbi.nlm.nih.gov/path to ftp://ftp.ncbi.nlm.nih.gov/path
        return http_ftp_path.replace("https://", "ftp://")
    elif http_ftp_path.startswith("http://ftp.ncbi.nlm.nih.gov/"):
        # Convert http://ftp.ncbi.nlm.nih.gov/path to ftp://ftp.ncbi.nlm.nih.gov/path
        return http_ftp_path.replace("http://", "ftp://")
    elif http_ftp_path.startswith("ftp://"):
        return http_ftp_path
    else:
        # If it doesn't start with a protocol, assume it's a path that needs ftp:// prefix
        if http_ftp_path.startswith("/"):
            return f"ftp://ftp.ncbi.nlm.nih.gov{http_ftp_path}"
        else:
            return f"ftp://ftp.ncbi.nlm.nih.gov/{http_ftp_path}"

# Create download URLs for md5checksum files with unique filenames using FTP protocol
md5_urls = []
url_to_filename = {}  # Map URLs to their unique filenames
filename_to_ftp_path = {}  # Map filenames back to FTP paths

for i, ftp_path in enumerate(mini_genebank["ftp_path"]):  # [:100] Test with first 100 entries - 3.3s, quick, so running on all
    if ftp_path and ftp_path != "na":
        # Convert to proper FTP URL and append md5checksums.txt
        ftp_url = convert_to_ftp_url(ftp_path)
        md5_url = ftp_url + "/md5checksums.txt"
        
        # Create unique filename using index to avoid conflicts
        unique_filename = f"md5checksums_{i:06d}.txt"
        
        md5_urls.append(md5_url)
        url_to_filename[md5_url] = unique_filename
        filename_to_ftp_path[unique_filename] = ftp_path

print(f"Preparing to download {len(md5_urls)} MD5 checksum files in parallel using FTP protocol")
print(f"Sample FTP URL: {md5_urls[0] if md5_urls else 'No URLs generated'}")

# FTP-optimized aria2 options
ftp_options = {
    "dir": temp_dir,
    "max-connection-per-server": "10",  # FTP servers often limit connections
    "split": "1",  # Don't split small text files
    "ftp-reuse-connection": "true",  # Reuse FTP connections for efficiency
    "ftp-pasv": "true",  # Use passive FTP mode (better for firewalls)
    "timeout": "60",  # 60 second timeout for FTP connections
    "retry-wait": "3",  # Wait 3 seconds between retries
    "max-tries": "3",  # Try up to 3 times per file
    "connect-timeout": "15",  # 30 second connection timeout
    "max-concurrent-downloads": "10",  # Limit total concurrent downloads
}

# Queue all downloads with progress bar
downloads = []
print("Queuing FTP downloads...")
for url in tqdm(md5_urls, desc="Queuing FTP downloads"):
    try:
        unique_filename = url_to_filename[url]
        download_options = ftp_options.copy()
        download_options["out"] = unique_filename  # Specify unique output filename
        download = api.add_uris([url], options=download_options)
        downloads.append((url, download, unique_filename))
    except Exception as e:
        print(f"Failed to queue download for {url}: {e}")

print(f"Queued {len(downloads)} FTP downloads. Waiting for aria2 to complete all downloads...")

Preparing to download 242833 MD5 checksum files in parallel using FTP protocol
Sample FTP URL: ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/018/101/015/GCA_018101015.1_ASM1810101v1/md5checksums.txt
Queuing FTP downloads...


Queuing FTP downloads:   0%|          | 0/242833 [00:00<?, ?it/s]

Queued 242833 FTP downloads. Waiting for aria2 to complete all downloads...


In [177]:
test_df = pl.scan_csv(temp_dir+"/*.txt",  separator=" ", include_file_paths="file_path", has_header=False,schema={"md5":pl.Utf8,"dropme":pl.Boolean,"file_path_on_ftp":pl.Utf8}
).collect()
test_df = test_df.drop("dropme")
test_df

md5,file_path_on_ftp,file_path
str,str,str
"""ba7bb432c803cec66a415cf43ae54e…","""./annotation_hashes.txt""","""/tmp/tmpqrl3mdmm/md5checksums_…"
"""d364bab756fa59130f83f23bface8f…","""./GCA_018101015.1_ASM1810101v1…","""/tmp/tmpqrl3mdmm/md5checksums_…"
"""f501bb76bb9b9d7bf21d5acf8c3731…","""./GCA_018101015.1_ASM1810101v1…","""/tmp/tmpqrl3mdmm/md5checksums_…"
"""1aeebbbf8a82ce15c162d3cc430120…","""./GCA_018101015.1_ASM1810101v1…","""/tmp/tmpqrl3mdmm/md5checksums_…"
"""3eb62af3551174adf510b43c50973a…","""./GCA_018101015.1_ASM1810101v1…","""/tmp/tmpqrl3mdmm/md5checksums_…"
…,…,…
"""e77fa231223ef22f88e9495eee6cdf…","""./GCA_003313675.1_ASM331367v1_…","""/tmp/tmpqrl3mdmm/md5checksums_…"
"""eec643dfd353493c38d4700bdee223…","""./GCA_003313675.1_ASM331367v1_…","""/tmp/tmpqrl3mdmm/md5checksums_…"
"""448bf4f1004acef74ebd35ca0590cf…","""./GCA_003313675.1_ASM331367v1_…","""/tmp/tmpqrl3mdmm/md5checksums_…"


In [102]:
# # Wait for all aria2 downloads to complete
# # Check aria2 global stats to see when all downloads are done
# for thing in tqdm(enumerate(downloads), desc="waiting for downloads to complete"):
#     stats = api.get_stats()
#     active = int(stats.num_active)
#     waiting = int(stats.num_waiting)
    
#     if active == 0 and waiting == 0:
#         print("All downloads completed!")
#         break
#     tqdm.update()
#     time.sleep(2)
        

In [178]:
# Now read all the downloaded files
ftp_path_md5_content = {}
completed_count = 0
failed_count = 0
failed_exmp =""
print("Processing downloaded files...")
for url, download, unique_filename in tqdm(downloads, desc="Reading downloaded files"):
    file_path = os.path.join(temp_dir, unique_filename)
    
    if os.path.exists(file_path):
        try:
            with open(file_path, 'r') as f:
                content = f.read()
            
            # Map FTP path to content using our mapping
            ftp_path = filename_to_ftp_path[unique_filename]
            ftp_path_md5_content[ftp_path] = content
            completed_count += 1
        except Exception as e:
            print(f"Error reading downloaded file {file_path}: {e}")
            failed_count += 1
            # print(url)
    else:
        # print(f"Downloaded file not found: {file_path}")
        failed_count += 1
        failed_exmp = url
        # print(url)

# # Clean up temporary files
# try:
#     import shutil
#     shutil.rmtree(temp_dir)
# except Exception as e:
#     print(f"Failed to clean up temp directory: {e}")

print(f"\n✅ Successfully processed {completed_count} MD5 checksum files")
print(f"❌ Failed/missing files: {failed_count}")
print(f"❌ Failed/missing example url: {failed_exmp}")
print(f"📊 Total attempted: {len(md5_urls)}")
print(f"📋 Sample of first entry: {list(ftp_path_md5_content.values())[0][:200] if ftp_path_md5_content else 'No content'}")
print(f"🔗 Mapping verification: {len(ftp_path_md5_content)} entries in final dictionary")

Processing downloaded files...


Reading downloaded files:   0%|          | 0/242833 [00:00<?, ?it/s]


✅ Successfully processed 60579 MD5 checksum files
❌ Failed/missing files: 182254
❌ Failed/missing example url: ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/038/506/625/GCA_038506625.1_ASM3850662v1/md5checksums.txt
📊 Total attempted: 242833
📋 Sample of first entry: ba7bb432c803cec66a415cf43ae54ef6  ./annotation_hashes.txt
d364bab756fa59130f83f23bface8fc1  ./GCA_018101015.1_ASM1810101v1_assembly_report.txt
f501bb76bb9b9d7bf21d5acf8c373195  ./GCA_018101015.1_ASM18
🔗 Mapping verification: 60579 entries in final dictionary


In [ ]:
mini_genebank = mini_genebank.with_columns(
        (pl.col("ftp_path") + "/md5checksums.txt").alias("md5sums_path")
)


In [ ]:
glob_path = "/genomes/all/GCA/001/974/195/*/" + "md5checksums.txt"

# fsspec.FTPFileSystem requires a host to be specified, which it infers from the path or uses the default.
# For this specific FTP server, a direct glob might be complex due to the anonymous access and path structure.
# A more robust solution might involve listing the subdirectories first.

# Let's try to get a list of subdirectories first
subdirs = fs.ls("/genomes/all/GCA/001/974/195/", detail=False)

# Filter for directories (they typically don't end in / with fs.ls)
# The sub-directory names will be GCA_001974195.1, GCA_001974195.2, etc.
file_paths = []
for path in subdirs:
    # Construct the full path to the md5checksums.txt file
    file_paths.append(path + "/md5checksums.txt")
    
# Now, construct the list of complete FTP URLs
# Since it's NCBI FTP, the host is 'ftp.ncbi.nlm.nih.gov'
host = "ftp.ncbi.nlm.nih.gov"
full_urls = [f"ftp://{host}{path}" for path in file_paths]

In [ ]:

minimini= mini_genebank[:10]
test_df = [pl.scan_csv(md5sums_path, sep="\t", has_header=False) for md5sums_path in minimini["md5sums_path"].to_list()]

In [34]:
len(ftp_path_md5_content)

99

In [ ]:
# Merge SILVA sequences and apply entropy masking
print("Merging and masking SILVA sequences")
silva_merged = os.path.join(rrna_dir, "SILVA_merged.fasta")
silva_masked = os.path.join(rrna_dir, "SILVA_merged_masked.fasta")

# Concatenate SILVA files
run_command_comp(
    base_cmd="cat",
    positional_args=[silva_ssu_path, silva_lsu_path],
    positional_args_location="end",
    params={},
    output_file=silva_merged,
    logger=logger,
)

# Apply entropy masking
bbduk(
    in1=silva_merged,
    out=silva_masked,
    entropy=0.6,
    entropyk=4,
    entropywindow=24,
    maskentropy=True,
    ziplevel=9,
)

print(f"Created masked SILVA rRNA database: {silva_masked}")

# clean up
try:
    os.remove(deduplicated_fasta)
    os.remove(compressed_path)
except Exception as e:
    logger.warning(f"Could not remove intermediate files: {e}")

print(f"Masking sequences prepared in {masking_dir}")

# ##### Create rRNA DB #####
# cd $rolypoly/data/
# mkdir rRNA
# cd rRNA
# gzip SILVA_138.1_SSURef_NR99_tax_silva.fasta.gz
# gzip SILVA_138.1_LSURef_NR99_tax_silva.fasta.gz
# cat *fasta > merged.fas

# bbduk.sh -Xmx1g in=merged.fas out=merged_masked.fa zl=9 entropy=0.6 entropyk=4 entropywindow=24 maskentropy
# bbduk.sh -Xmx1g in=rmdup_rRNA_ncbi.fasta  out=rmdup_rRNA_ncbi_masked.fa zl=9 entropy=0.6 entropyk=4 entropywindow=24 maskentropy


## rRNAs, tRNA, mtDNA, and plastid-DNA from NCBI

rRNA from NCBI nuc X refseq

In [14]:
# %%bash
# search_term="((ribosomal RNA[Title]) OR rRNA[Title]) AND 150:7000[Sequence Length] AND txid131567[Organism:exp] "
# # Perform the search and download the sequences
# esearch -db nuccore -query "$search_term" | efetch -format fasta > "rrna_genes_refseq.fasta"

curl: (22) The requested URL returned error: 400 Bad Request
 ERROR:  curl command failed ( Mon Dec  1 15:02:00 PST 2025 ) with: 22
-X POST https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi -d query_key=1&WebEnv=MCID_692e1ba5bfc86b4c4b0614df&retstart=18200&retmax=50&db=nuccore&rettype=fasta&retmode=text&tool=edirect&edirect=24.0&edirect_os=Linux&email=uneri%40ln008.jgi
nquire -url https://eutils.ncbi.nlm.nih.gov/entrez/eutils/ efetch.fcgi -query_key 1 -WebEnv MCID_692e1ba5bfc86b4c4b0614df -retstart 18200 -retmax 50 -db nuccore -rettype fasta -retmode text -tool edirect -edirect 24.0 -edirect_os Linux -email uneri@ln008.jgi
EMPTY RESULT
SECOND ATTEMPT
curl: (22) The requested URL returned error: 400 Bad Request
 ERROR:  curl command failed ( Mon Dec  1 15:03:03 PST 2025 ) with: 22
-X POST https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi -d query_key=1&WebEnv=MCID_692e1ba5bfc86b4c4b0614df&retstart=19700&retmax=50&db=nuccore&rettype=fasta&retmode=text&tool=edirect&edirect

tRNA from NCBI nuc

In [144]:
# first, getting the contents of https://ftp.ncbi.nlm.nih.gov/blast/db/ and filtering to files containing "16S", "18S", "28S", "ITS" 
# LSU, SSU
import ftplib
from ftplib import FTP

terms = ["16S", "18S", "28S", "ITS", "LSU", "SSU", "ribosomal"]
files_2_download = []
FTP_HOST = "ftp.ncbi.nlm.nih.gov"
FTP_DIR = "/blast/db/"

# List to store the file and directory names
file_list = []

# Connect to the FTP host
with FTP(FTP_HOST) as ftp:
    #  Log in anonymously (default behavior for .login())
    ftp.login()
    print(f"Connected to {FTP_HOST}")
    ftp.cwd(FTP_DIR)
    logger.debug(f"Changed directory to {FTP_DIR}")

    # nlst() returns only names, not detailed information
    file_list = ftp.nlst()
    
    print("\n--- Directory Contents ---")
    for item in file_list:
        if any(term in item for term in terms):
            files_2_download.append(item)
            logger.debug(item)
            
            # Download file directly using FTP
            local_path = os.path.join(rrna_dir, item)
            print(f"Downloading {item}")
            with open(local_path, 'wb') as local_file:
                ftp.retrbinary(f'RETR {item}', local_file.write)
            logger.debug(f"Successfully downloaded {item}")

print(f"Downloaded {len(files_2_download)} files matching terms: {terms}")

Connected to ftp.ncbi.nlm.nih.gov

--- Directory Contents ---

--- Directory Contents ---
Downloaded 24 files matching terms: ['16S', '18S', '28S', 'ITS', 'LSU', 'SSU', 'ribosomal']
Downloaded 24 files matching terms: ['16S', '18S', '28S', 'ITS', 'LSU', 'SSU', 'ribosomal']


In [167]:
# for each of these, will run blastdbcmd to extract sequences into fasta 
from rolypoly.utils.various import extract
import subprocess as sp
for item in files_2_download:
    if not item.endswith('.tar.gz'):
        continue
    print(f"Processing {item}...")
    extract(archive_path=os.path.join(rrna_dir, item),
        extract_to=rrna_dir,
    )
    # blastdbcmd -entry all -db LSU_prokaryote_rRNA  -out reference.fasta -outfmt "%T;%t;%s" # taxid;header;sequence
    run_command_comp(
        base_cmd="blastdbcmd",
        positional_args=[
            "-entry",
            "all",
            "-db",
            os.path.join(rrna_dir, item.replace('.tar.gz','')),
            "-out",
            os.path.join(rrna_dir, item.replace('.tar.gz','').replace('.tar','') + '.tab'),
            "-outfmt",
            str(f'"%T@%t@%K@%s"') ,
        ],
        positional_args_location="end",
        params={},
        logger=logger,
        output_file=os.path.join(rrna_dir, item.replace('.tar.gz','').replace('.tar','') + '.tab')
    )


Processing 18S_fungal_sequences.tar.gz...


Successfully processed 
'/clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/18S_fungal_sequences.tar.gz' to 
'/clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna'

INFO     2025-12-02 18:51:53 - Running command: blastdbcmd   -entry all -db                                                            ]8;id=969219;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py\various.py]8;;\:]8;id=181260;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py#1257\1257]8;;\
         /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/18S_fungal_sequences -out                                         
         /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/18S_fungal_sequences.tab -outfmt "%T@%t@%K@%s"                    

File '/clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/18S_fungal_sequences.tab' size is 
5700653 bytes (not empty). 

Processing 28S_fungal_sequences.tar.gz...


...

File '/clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/SSU_eukaryote_rRNA.tab' size is 
16578954 bytes (not empty). 

Processing 16S_ribosomal_RNA.tar.gz...


Successfully processed 
'/clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/16S_ribosomal_RNA.tar.gz' to 
'/clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna'

INFO     2025-12-02 18:52:24 - Running command: blastdbcmd   -entry all -db                                                            ]8;id=267176;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py\various.py]8;;\:]8;id=995770;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py#1257\1257]8;;\
         /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/16S_ribosomal_RNA -out                                            
         /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/16S_ribosomal_RNA.tab -outfmt "%T@%t@%K@%s"                       

File '/clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/16S_ribosomal_RNA.tab' size is 
42549204 bytes (not empty). 

In [168]:
import glob
# get all files to remove (everything but the .tab files)
all_files_in_rrna = glob.glob(os.path.join(rrna_dir, "*"))
all_tab_files = glob.glob(os.path.join(rrna_dir, "*.tab"))
all_fasta_files = glob.glob(os.path.join(rrna_dir, "*.fasta"))

# Files to keep (tab files, fasta files, and parquet/tsv files)
files_to_keep = set(all_tab_files + all_fasta_files + 
                   glob.glob(os.path.join(rrna_dir, "*.parquet")) +
                   glob.glob(os.path.join(rrna_dir, "*.tsv")) +
                   glob.glob(os.path.join(rrna_dir, "*.gz")))

# Get all files to remove (exclude directories and files to keep)
files_2_remove = [f for f in all_files_in_rrna if os.path.isfile(f) and f not in files_to_keep]

print(f"Found {len(files_2_remove)} files to remove")
print(f"Keeping {len(files_to_keep)} files (.tab, .fasta, .parquet, .tsv, .gz)")

# Remove the files
for file_path in files_2_remove:
    try:
        os.remove(file_path)
        print(f"Removed: {os.path.basename(file_path)}")
    except Exception as e:
        print(f"Failed to remove {os.path.basename(file_path)}: {e}")

print("Cleanup complete")

Found 83 files to remove
Keeping 28 files (.tab, .fasta, .parquet, .tsv, .gz)
Removed: 16S_ribosomal_RNA.ndb
Removed: 16S_ribosomal_RNA.nhr
Removed: 16S_ribosomal_RNA.nin
Removed: 16S_ribosomal_RNA.nnd
Removed: 16S_ribosomal_RNA.nni
Removed: 16S_ribosomal_RNA.nog
Removed: 16S_ribosomal_RNA.nos
Removed: 16S_ribosomal_RNA.not
Removed: 16S_ribosomal_RNA.nsq
Removed: 16S_ribosomal_RNA.ntf
Removed: 16S_ribosomal_RNA.nto
Removed: 18S_fungal_sequences.ndb
Removed: 18S_fungal_sequences.nhr
Removed: 18S_fungal_sequences.nin
Removed: 18S_fungal_sequences.nnd
Removed: 18S_fungal_sequences.nni
Removed: 18S_fungal_sequences.nog
Removed: 18S_fungal_sequences.nos
Removed: 18S_fungal_sequences.not
Removed: 18S_fungal_sequences.nsq
Removed: 18S_fungal_sequences.ntf
Removed: 18S_fungal_sequences.nto
Removed: 28S_fungal_sequences.ndb
Removed: 28S_fungal_sequences.nhr
Removed: 28S_fungal_sequences.nin
Removed: 28S_fungal_sequences.nnd
Removed: 28S_fungal_sequences.nni
Removed: 28S_fungal_sequences.nog
Rem

In [202]:
rrna_df = pl.scan_csv(
    os.path.join(rrna_dir, "*.tab"),
    separator="@",
    has_header=False,
    null_values = ["N/A"],
    new_columns=["taxid", "header","name","sequence"],
    include_file_paths="type",
).collect()
print(f"Read {rrna_df.height} rRNA sequences from {tab_path}")

Read 159996 rRNA sequences from /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/18S_fungal_sequences.tab


In [203]:
# rrna_df["name"].value_counts(sort=True) # none?
rrna_df = rrna_df.drop("name")

In [204]:
rrna_df = rrna_df.with_columns(
    rRNA_type=pl.col("type").str.extract(r"([^/]+)\.tab$", 1)
)

rrna_df = rrna_df.drop("type")
rrna_df["rRNA_type"].value_counts(sort = True)


rRNA_type,count
str,u32
"""ITS_eukaryote_sequences""",77582
"""16S_ribosomal_RNA""",27708
"""ITS_RefSeq_Fungi""",19541
"""28S_fungal_sequences""",11754
"""SSU_eukaryote_rRNA""",8784
"""LSU_eukaryote_rRNA""",6575
"""LSU_prokaryote_rRNA""",4047
"""18S_fungal_sequences""",4005


In [205]:
from rolypoly.utils.bio.sequences import write_fasta_file,remove_duplicates
rrna_fasta_path = os.path.join(rrna_dir, "ncbi_rRNA_all_sequences.fasta")

write_fasta_file(
    seqs=rrna_df["sequence"].to_list(),
    headers=rrna_df["header"].to_list(),
    output_file=rrna_fasta_path,
)
print(f"Wrote combined rRNA fasta to {rrna_fasta_path}")

Wrote combined rRNA fasta to /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/ncbi_rRNA_all_sequences.fasta


In [214]:
mini_genebank["group"].value_counts(sort=True)

group,count
str,u32
"""bacteria""",125828
"""viral""",88342
"""fungi""",7870
"""invertebrate""",7069
"""vertebrate_other""",4354
…,…
"""plant""",3385
"""vertebrate_mammalian""",1124
"""protozoa""",1123


In [215]:
temp_df = rrna_df.join(
    mini_genebank.select(
        ["ncbi_taxonid", "organism_name"]
    ).unique(),  # seq_length
    left_on=["taxid"],
    right_on=["ncbi_taxonid"],
    how="left",
)
temp_df["organism_name"].value_counts(sort=True)


organism_name,count
str,u32
null,117964
"""Giardia duodenalis""",127
"""Globisporangium heterothallicu…",93
"""Babesia bovis""",65
"""Globisporangium sylvaticum""",61
…,…
"""Andreaea rupestris""",1
"""Tetraphis pellucida""",1
"""Anncaliia algerae""",1


## NCBI taxdump

In [216]:
# *.dmp files are bcp-like dump from GenBank taxonomy database.

# General information.
# Field terminator is "\t|\t"
# Row terminator is "\t|\n"

# nodes.dmp file consists of taxonomy nodes. The description for each node includes the following
# fields:
# 	tax_id					-- node id in GenBank taxonomy database
#  	parent tax_id				-- parent node id in GenBank taxonomy database
#  	rank					-- rank of this node (domain, kingdom, ...) 
#  	embl code				-- locus-name prefix; not unique
#  	division id				-- see division.dmp file
#  	inherited div flag  (1 or 0)		-- 1 if node inherits division from parent
#  	genetic code id				-- see gencode.dmp file
#  	inherited GC  flag  (1 or 0)		-- 1 if node inherits genetic code from parent
#  	mitochondrial genetic code id		-- see gencode.dmp file
#  	inherited MGC flag  (1 or 0)		-- 1 if node inherits mitochondrial gencode from parent
#  	GenBank hidden flag (1 or 0)            -- 1 if name is suppressed in GenBank entry lineage
#  	hidden subtree root flag (1 or 0)       -- 1 if this subtree has no sequence data yet
#  	comments				-- free-text comments and citations

# Taxonomy names file (names.dmp):
# 	tax_id					-- the id of node associated with this name
# 	name_txt				-- name itself
# 	unique name				-- the unique variant of this name if name not unique
# 	name class				-- (synonym, common name, ...)
fetch_and_extract(
    url="https://ftp.ncbi.nlm.nih.gov/pub/taxonomy/taxdump.tar.gz",
    fetched_to=os.path.join(data_dir, "taxdump.tar.gz"),
    extract_to=os.path.join(data_dir, "taxdump"),
    logger=logger,
)


INFO     2025-12-02 19:16:59 - Downloading https://ftp.ncbi.nlm.nih.gov/pub/taxonomy/taxdump.tar.gz to                                  ]8;id=203324;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py\various.py]8;;\:]8;id=200555;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py#215\215]8;;\
         /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/taxdump.tar.gz                                                                

INFO     2025-12-02 19:17:06 - Successfully downloaded to /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/taxdump.tar.gz ]8;id=382276;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py\various.py]8;;\:]8;id=849077;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py#221\221]8;;\

INFO     2025-12-02 19:17:06 - Extracting /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/taxdump.tar.gz to              ]8;id=127845;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py\various.py]8;;\:]8;id=832875;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py#255\255]8;;\
         /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/taxdump                                                                       

INFO     2025-12-02 19:17:09 - Successfully extracted to /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/taxdump         ]8;id=591732;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py\various.py]8;;\:]8;id=279401;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py#288\288]8;;\

PosixPath('/clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/taxdump')

In [222]:
len(fields)

12

In [ ]:
taxonomy_dir = os.path.join(data_dir, "taxdump")
nodes_path = os.path.join(taxonomy_dir, "nodes.dmp")
names_path = os.path.join(taxonomy_dir, "names.dmp")
# can not read with polars cause of irregular separator, so using line by line read...

records = []
with open(nodes_path, "r") as f:
    nodes_lines = f.readlines()
    for line in tqdm(nodes_lines):
        line = line.rstrip("\n").rstrip("\t|")
        fields = line.split("\t|\t")
        record = {
            "tax_id": fields[0],
            "parent_tax_id": fields[1],
            "rank": fields[2],
            "embl_code": fields[3],
            "division_id": fields[4],
            "inherited_div_flag": fields[5],
            "genetic_code_id": fields[6],
            "inherited_GC_flag": fields[7],
            "mitochondrial_genetic_code_id": fields[8],
            "inherited_MGC_flag": fields[9],
            "GenBank_hidden_flag": fields[10],
            "hidden_subtree_root_flag": fields[11],
            # "comments": fields[12]
        }
        records.append(record)
nodes_df = pl.from_records(records).with_columns(
    pl.col("tax_id").cast(pl.Int64),
    pl.col("parent_tax_id").cast(pl.Int64),
    
)
nodes_df
#


  0%|          | 0/2708826 [00:00<?, ?it/s]

tax_id,parent_tax_id,rank,embl_code,division_id,inherited_div_flag,genetic_code_id,inherited_GC_flag,mitochondrial_genetic_code_id,inherited_MGC_flag,GenBank_hidden_flag,hidden_subtree_root_flag
i64,i64,str,str,str,str,str,str,str,str,str,str
1,1,"""no rank""","""""","""8""","""0""","""1""","""0""","""0""","""0""","""0""","""0"""
2,131567,"""domain""","""""","""0""","""0""","""11""","""0""","""0""","""0""","""0""","""0"""
6,335928,"""genus""","""""","""0""","""1""","""11""","""1""","""0""","""1""","""0""","""0"""
7,6,"""species""","""AC""","""0""","""1""","""11""","""1""","""0""","""1""","""1""","""0"""
9,32199,"""species""","""BA""","""0""","""1""","""11""","""1""","""0""","""1""","""1""","""0"""
…,…,…,…,…,…,…,…,…,…,…,…
3475891,47833,"""no rank""","""""","""1""","""1""","""1""","""1""","""5""","""1""","""0""","""0"""
3476007,479661,"""varietas""","""RC""","""4""","""1""","""1""","""1""","""1""","""1""","""0""","""0"""
3476068,2653920,"""genus""","""""","""4""","""1""","""1""","""1""","""1""","""1""","""0""","""0"""


In [226]:
records = []
with open(names_path, "r") as f:
    nodes_lines = f.readlines()
    for line in tqdm(nodes_lines):
        line = line.rstrip("\n").rstrip("\t|")
        fields = line.split("\t|\t")
        record = {
            "tax_id": fields[0],
            "name_txt": fields[1],
            "unique_name": fields[2],
            "name_class": fields[3],
        }
        records.append(record)
names_df = pl.from_records(records).with_columns(
    pl.col("tax_id").cast(pl.Int64),
    
)
names_df

  0%|          | 0/4495871 [00:00<?, ?it/s]

tax_id,name_txt,unique_name,name_class
i64,str,str,str
1,"""all""","""""","""synonym"""
1,"""root""","""""","""scientific name"""
2,"""Bacteria""","""Bacteria <bacteria>""","""scientific name"""
2,"""bacteria""","""bacteria <blast name>""","""blast name"""
2,"""bacteria""","""bacteria <genbank common name>""","""genbank common name"""
…,…,…,…
3476068,"""Comoliopsis Wurdack, 1984""","""""","""authority"""
3476159,"""Benthoplanidae Bezio, Paulay &…","""""","""authority"""
3476159,"""Benthoplanidae""","""""","""scientific name"""


In [ ]:
# test = names_df.sample(1) 
# for col_name in names_df.columns:
#     print(f"{col_name}: {test[col_name].first()}")



tax_id: 29018
name_txt: Centruroides margaritatus (Gervais, 1841)
unique_name: 
name_class: authority


In [262]:
# now let's form the lineage strings
sci_name_df = names_df.filter(pl.col("name_class") == "scientific name").select(
    ["tax_id", "name_txt"]  ).rename({"name_txt": "scientific_name"}).unique()
# sci_name_df.null_count()

In [263]:
def build_all_lineages_efficient(nodes_df, sci_name_df):
    """Build lineage strings for all tax_ids efficiently using polars operations"""
    
    # Join nodes with scientific names to get name for each tax_id
    nodes_with_names = nodes_df.join(
        sci_name_df, 
        on="tax_id", 
        how="left"
    ).select(["tax_id", "parent_tax_id", "scientific_name"])
    
    # Build a dictionary for faster lookups
    tax_to_parent = dict(zip(
        nodes_with_names["tax_id"].to_list(),
        nodes_with_names["parent_tax_id"].to_list()
    ))
    tax_to_name = dict(zip(
        nodes_with_names["tax_id"].to_list(),
        nodes_with_names["scientific_name"].to_list()
    ))
    
    def get_lineage(tax_id):
        """Get lineage for a single tax_id"""
        if tax_id not in tax_to_name or tax_to_name[tax_id] is None:
            return ""
            
        lineage = []
        current = tax_id
        visited = set()
        
        while current in tax_to_parent and current not in visited and current != 1:
            visited.add(current)
            name = tax_to_name.get(current)
            if name:
                lineage.append(name)
            
            parent = tax_to_parent[current]
            if parent == current:  # Root case
                break
            current = parent
        
        return ";".join(reversed(lineage))
    
    # Get all unique tax_ids
    all_tax_ids = sci_name_df["tax_id"].unique().sort().to_list()
    print(f"Building lineages for {len(all_tax_ids)} tax_ids...")
    
    # Build lineages
    lineages = []
    for tax_id in tqdm(all_tax_ids):
        lineage = get_lineage(tax_id)
        lineages.append(lineage)
    
    # Create result DataFrame
    result_df = pl.DataFrame({
        "tax_id": all_tax_ids,
        "lineage": lineages
    })
    
    return result_df

# Build all lineages
lineage_df = build_all_lineages_efficient(nodes_df, sci_name_df)
print(f"Built {lineage_df.height} lineage strings")

# Show some examples with non-empty lineages
non_empty_lineages = lineage_df.filter(pl.col("lineage") != "")
print(f"\nFound {non_empty_lineages.height} non-empty lineages")
print("\nExample lineages:")
print(non_empty_lineages.head(10))

Building lineages for 2708826 tax_ids...


100%|██████████| 2708826/2708826 [00:18<00:00, 149142.03it/s]


Built 2708826 lineage strings

Found 2708825 non-empty lineages

Example lineages:
shape: (10, 2)
┌────────┬─────────────────────────────────┐
│ tax_id ┆ lineage                         │
│ ---    ┆ ---                             │
│ i64    ┆ str                             │
╞════════╪═════════════════════════════════╡
│ 2      ┆ cellular organisms;Bacteria     │
│ 6      ┆ cellular organisms;Bacteria;Ps… │
│ 7      ┆ cellular organisms;Bacteria;Ps… │
│ 9      ┆ cellular organisms;Bacteria;Ps… │
│ 10     ┆ cellular organisms;Bacteria;Ps… │
│ 11     ┆ cellular organisms;Bacteria;Ba… │
│ 13     ┆ cellular organisms;Bacteria;Ps… │
│ 14     ┆ cellular organisms;Bacteria;Ps… │
│ 16     ┆ cellular organisms;Bacteria;Ps… │
│ 17     ┆ cellular organisms;Bacteria;Ps… │
└────────┴─────────────────────────────────┘


In [264]:
# Efficient tree structures for taxonomic neighbor finding
from collections import defaultdict, deque
import heapq

class TaxonomyTree:
    """
    Efficient taxonomy tree for finding nearest neighbors with available data.
    
    This implementation uses:
    - Dictionary-based adjacency lists for O(1) parent/child lookups
    - BFS for finding nearest ancestors/descendants with data
    - Caching for repeated queries
    """
    
    def __init__(self, nodes_df, data_availability_df=None):
        """
        Initialize taxonomy tree.
        
        Args:
            nodes_df: DataFrame with tax_id, parent_tax_id, rank, scientific_name
            data_availability_df: DataFrame with tax_id and availability flags
                                (e.g., has_genome, has_transcriptome, has_assembly)
        """
        self.nodes_df = nodes_df
        self.parent_map = {}  # tax_id -> parent_tax_id
        self.children_map = defaultdict(list)  # parent_tax_id -> [child_tax_ids]
        self.rank_map = {}  # tax_id -> rank
        self.name_map = {}  # tax_id -> scientific_name
        self.data_available = set()  # tax_ids with available data
        self.cache = {}  # Cache for repeated queries
        
        self._build_tree()
        if data_availability_df is not None:
            self._set_data_availability(data_availability_df)
    
    def _build_tree(self):
        """Build internal tree structure from nodes DataFrame"""
        for row in self.nodes_df.iter_rows(named=True):
            tax_id = row['tax_id']
            parent_tax_id = row['parent_tax_id']
            
            self.parent_map[tax_id] = parent_tax_id
            self.children_map[parent_tax_id].append(tax_id)
            
            # Store additional info if available
            if 'rank' in row:
                self.rank_map[tax_id] = row['rank']
            if 'scientific_name' in row:
                self.name_map[tax_id] = row['scientific_name']
    
    def _set_data_availability(self, data_df):
        """Set which tax_ids have available data"""
        # Assume any non-null, non-empty columns indicate data availability
        available_tax_ids = []
        
        for row in data_df.iter_rows(named=True):
            tax_id = row['tax_id']
            # Check if any data columns are available
            has_data = any(
                row.get(col) and row.get(col) not in ['', 'na', 'NA', '-', None]
                for col in data_df.columns 
                if col != 'tax_id'
            )
            if has_data:
                available_tax_ids.append(tax_id)
        
        self.data_available = set(available_tax_ids)
        print(f"Set data availability for {len(self.data_available)} tax_ids")
    
    def find_nearest_with_data(self, tax_id, max_distance=10):
        """
        Find nearest taxonomic neighbor(s) with available data.
        
        Args:
            tax_id: Starting taxonomy ID
            max_distance: Maximum taxonomic distance to search
        
        Returns:
            dict with 'ancestor', 'descendants', and 'distance' info
        """
        if tax_id in self.cache:
            return self.cache[tax_id]
        
        if tax_id in self.data_available:
            result = {
                'ancestor': {'tax_id': tax_id, 'distance': 0, 'name': self.name_map.get(tax_id)},
                'descendants': [],
                'self_has_data': True
            }
            self.cache[tax_id] = result
            return result
        
        # BFS to find nearest ancestor with data
        nearest_ancestor = self._find_nearest_ancestor_with_data(tax_id, max_distance)
        
        # BFS to find nearest descendants with data  
        nearest_descendants = self._find_nearest_descendants_with_data(tax_id, max_distance)
        
        result = {
            'ancestor': nearest_ancestor,
            'descendants': nearest_descendants,
            'self_has_data': False
        }
        
        self.cache[tax_id] = result
        return result
    
    def _find_nearest_ancestor_with_data(self, tax_id, max_distance):
        """Find nearest ancestor with data using BFS up the tree"""
        current = tax_id
        distance = 0
        
        while current in self.parent_map and distance <= max_distance:
            parent = self.parent_map[current]
            distance += 1
            
            if parent in self.data_available:
                return {
                    'tax_id': parent, 
                    'distance': distance,
                    'name': self.name_map.get(parent),
                    'rank': self.rank_map.get(parent)
                }
            
            if parent == current:  # Root node
                break
            current = parent
        
        return None
    
    def _find_nearest_descendants_with_data(self, tax_id, max_distance):
        """Find nearest descendants with data using BFS down the tree"""
        if tax_id not in self.children_map:
            return []
        
        # BFS queue: (tax_id, distance)
        queue = deque([(child, 1) for child in self.children_map[tax_id]])
        descendants_with_data = []
        
        while queue:
            current_id, distance = queue.popleft()
            
            if distance > max_distance:
                break
            
            if current_id in self.data_available:
                descendants_with_data.append({
                    'tax_id': current_id,
                    'distance': distance,
                    'name': self.name_map.get(current_id),
                    'rank': self.rank_map.get(current_id)
                })
            
            # Add children to queue for further exploration
            if current_id in self.children_map:
                for child in self.children_map[current_id]:
                    queue.append((child, distance + 1))
        
        # Sort by distance
        return sorted(descendants_with_data, key=lambda x: x['distance'])
    
    def get_lineage_path(self, tax_id):
        """Get full lineage path from root to tax_id"""
        lineage = []
        current = tax_id
        visited = set()
        
        while current in self.parent_map and current not in visited:
            visited.add(current)
            lineage.append({
                'tax_id': current,
                'name': self.name_map.get(current),
                'rank': self.rank_map.get(current),
                'has_data': current in self.data_available
            })
            
            parent = self.parent_map[current]
            if parent == current:  # Root
                break
            current = parent
        
        return list(reversed(lineage))
    
    def find_common_ancestor(self, tax_id1, tax_id2):
        """Find lowest common ancestor of two tax_ids"""
        lineage1 = {node['tax_id']: i for i, node in enumerate(self.get_lineage_path(tax_id1))}
        lineage2_path = self.get_lineage_path(tax_id2)
        
        # Find first common node in lineage2 that's also in lineage1
        for node in lineage2_path:
            if node['tax_id'] in lineage1:
                return node
        
        return None

print("TaxonomyTree class defined for efficient neighbor finding")

TaxonomyTree class defined for efficient neighbor finding


In [265]:
# Example usage: Create taxonomy tree with your data
print("Creating TaxonomyTree with NCBI nodes...")

# Merge nodes with scientific names for the tree
nodes_with_names = nodes_df.join(
    sci_name_df, 
    on="tax_id", 
    how="left"
).select(["tax_id", "parent_tax_id", "rank", "scientific_name"])

# Create tree
tax_tree = TaxonomyTree(nodes_with_names)

# Set data availability based on your GenBank assembly data
# Filter to only include entries with actual genome/transcriptome data
data_available_df = mini_genebank.filter(
    # Filter for assemblies that actually have data
    (pl.col("ftp_path").is_not_null()) & 
    (pl.col("ftp_path") != "") & 
    (pl.col("ftp_path") != "na") &
    (pl.col("assembly_level").is_in(["Complete Genome", "Chromosome", "Scaffold", "Contig"]))
).select(["ncbi_taxonid"]).rename({"ncbi_taxonid": "tax_id"}).with_columns(
    pl.lit(True).alias("has_assembly")
)

print(f"Found {data_available_df.height} tax_ids with available assembly data")
tax_tree._set_data_availability(data_available_df)

print("TaxonomyTree ready for neighbor finding!")

Creating TaxonomyTree with NCBI nodes...
Found 242833 tax_ids with available assembly data
Set data availability for 242833 tax_ids
TaxonomyTree ready for neighbor finding!
Found 242833 tax_ids with available assembly data
Set data availability for 242833 tax_ids
TaxonomyTree ready for neighbor finding!


In [ ]:
# Test the tree with some example queries
print("Testing nearest neighbor finding...")

# Get some test tax_ids from your rRNA data that might not have assemblies
test_tax_ids = rrna_df["taxid"].cast(pl.Int64).unique().limit(5).to_list()
print(f"Testing with tax_ids: {test_tax_ids}")

for tax_id in test_tax_ids:
    print(f"\n--- Tax ID {tax_id} ---")
    try:
        result = tax_tree.find_nearest_with_data(tax_id, max_distance=5)
        
        if result['self_has_data']:
            print(f"✅ {tax_id} has data directly!")
        else:
            if result['ancestor']:
                ancestor = result['ancestor']
                print(f"🔼 Nearest ancestor with data: {ancestor['name']} (tax_id: {ancestor['tax_id']}, distance: {ancestor['distance']})")
            
            if result['descendants']:
                print(f"🔽 Descendants with data ({len(result['descendants'])}):")
                for desc in result['descendants'][:3]:  # Show first 3
                    print(f"   - {desc['name']} (tax_id: {desc['tax_id']}, distance: {desc['distance']})")
            
            if not result['ancestor'] and not result['descendants']:
                print(f"❌ No relatives with data found within distance 5")
                
    except Exception as e:
        print(f"❌ Error processing tax_id {tax_id}: {e}")

print("\nTree structure ready for production use!")

In [ ]:
# Alternative: Using NetworkX for more advanced tree operations
# NetworkX is excellent for complex graph algorithms and visualization

try:
    import networkx as nx
    
    def create_networkx_taxonomy_tree(nodes_df, data_availability_set):
        """Create NetworkX directed graph for taxonomy tree"""
        G = nx.DiGraph()
        
        # Add nodes with attributes
        for row in nodes_df.iter_rows(named=True):
            tax_id = row['tax_id']
            G.add_node(tax_id, 
                      parent=row['parent_tax_id'],
                      rank=row.get('rank', ''),
                      name=row.get('scientific_name', ''),
                      has_data=tax_id in data_availability_set)
            
            # Add edges (parent -> child relationships)
            parent_id = row['parent_tax_id']
            if parent_id != tax_id:  # Avoid self-loops
                G.add_edge(parent_id, tax_id)
        
        return G
    
    def find_nearest_data_nx(G, tax_id, max_distance=10):
        """Find nearest neighbors with data using NetworkX algorithms"""
        if not G.has_node(tax_id):
            return None
            
        if G.nodes[tax_id]['has_data']:
            return {'self': tax_id, 'distance': 0}
        
        # Find ancestors (predecessors in directed path to root)
        ancestors = []
        current = tax_id
        distance = 0
        while distance < max_distance:
            predecessors = list(G.predecessors(current))
            if not predecessors:
                break
            parent = predecessors[0]  # Should be only one parent
            distance += 1
            if G.nodes[parent]['has_data']:
                ancestors.append({'tax_id': parent, 'distance': distance, 
                                'name': G.nodes[parent]['name']})
                break
            current = parent
        
        # Find descendants using BFS
        descendants = []
        for node in nx.bfs_tree(G, tax_id, max_depth=max_distance):
            if node != tax_id and G.nodes[node]['has_data']:
                distance = nx.shortest_path_length(G, tax_id, node)
                descendants.append({'tax_id': node, 'distance': distance,
                                   'name': G.nodes[node]['name']})
        
        return {
            'ancestors': ancestors,
            'descendants': sorted(descendants, key=lambda x: x['distance'])
        }
    
    print("✅ NetworkX available - can use for advanced tree operations")
    print("   - Betweenness centrality analysis")
    print("   - Community detection")
    print("   - Tree visualization with graphviz")
    print("   - Shortest path algorithms")
    
except ImportError:
    print("⚠️  NetworkX not available - using custom TaxonomyTree class")
    print("   Install with: pip install networkx")

# Alternative libraries to consider:
print("\n📚 Other efficient tree libraries for Python:")
print("1. **anytree** - Simple tree data structure with search/filtering")
print("2. **treelib** - Tree data structure with pretty printing")  
print("3. **ete3** - Phylogenetic tree analysis (great for taxonomic trees)")
print("4. **igraph** - High-performance graph analysis")
print("5. **Bio.Phylo** (Biopython) - Phylogenetic tree I/O and analysis")

# For your use case, I recommend:
print("\n🎯 Recommendation for your use case:")
print("- **Custom TaxonomyTree** (above): Best performance for simple neighbor queries")
print("- **ete3**: Best for complex taxonomic operations and tree analysis")  
print("- **NetworkX**: Best for graph algorithms and community detection")
print("- **anytree**: Simple and intuitive API for basic tree operations")

In [ ]:
def prepare_plastid_data(data_dir, logger):
    """Prepare plastid sequence data for contamination filtering.

    Downloads NCBI RefSeq plastid sequences, combines them, and removes duplicates.

    Args:
        data_dir (str): Base directory for data storage
        logger: Logger object for recording progress and errors
    returns:
        None
    """
    plastid_dir = os.path.join(data_dir, "reference_seqs", "plastid_refseq")
    os.makedirs(plastid_dir, exist_ok=True)
    
    print("Downloading NCBI RefSeq plastid sequences")
    
    base_url = "https://ftp.ncbi.nlm.nih.gov/refseq/release/plastid/plastid."
    suffix = ".genomic.fna.gz"
    files_to_get = ["1.1", "1.2", "2.1", "2.2", "3.1"]
    
    all_files =[]
    downloaded_files = []
    
    for version in files_to_get:
        file_url = f"{base_url}{version}{suffix}"
        gz_filename = f"plastid.{version}.genomic.fna.gz"
        fasta_filename = f"plastid.{version}.genomic.fna"
        
        print(f"Downloading plastid version {version}")
        
        # Download and extract the file
        try:
            extracted_path = fetch_and_extract(
                url=file_url,
                fetched_to=os.path.join(plastid_dir, gz_filename),
                extract_to=plastid_dir,
                expected_file=fasta_filename,
                logger=logger,
            )
            downloaded_files.append(extracted_path)
            all_files.append(extracted_path)
            all_files.append(os.path.join(plastid_dir, gz_filename))
            print(f"Successfully downloaded and extracted {fasta_filename}")
        except Exception as e:
            logger.error(f"Failed to download plastid version {version}: {e}")
            continue
    
    if not downloaded_files:
        logger.error("No plastid files were successfully downloaded")
        return
    
    # Combine and deduplicate the sequences
    combined_fasta = os.path.join(plastid_dir, "combined_plastid_refseq.fasta")
    print(f"Combining and deduplicating {len(downloaded_files)} plastid files")
    
    stats = remove_duplicates(
        input_file=downloaded_files,
        output_file=combined_fasta,
        by="seq",
        revcomp_as_distinct=False,  # Treat reverse complement as duplicate
        return_stats=True,
        logger=logger,
    )
    
    if stats:
        print(
            f"Plastid deduplication stats: {stats['unique_records']} unique sequences from {stats['total_records']} total, {stats['duplicates_removed']} duplicates removed"
        )
    
    # Clean up individual files to save space (optional)
    try:
        for file_path in all_files:
            if os.path.exists(file_path):
                os.remove(file_path)
                logger.debug(f"Removed intermediate file: {os.path.basename(file_path)}")
    except Exception as e:
        logger.warning(f"Could not remove intermediate plastid files: {e}")
    
    print(f"Plastid sequences prepared in {plastid_dir}")


def prepare_mito_data(data_dir, logger):
    """Prepare mito sequence data for contamination filtering.

    Downloads NCBI RefSeq mito sequences, combines them, and removes duplicates.

    Args:
        data_dir (str): Base directory for data storage
        logger: Logger object for recording progress and errors
    returns:
        None
    """
    mito_dir = os.path.join(data_dir, "reference_seqs", "mito_refseq")
    os.makedirs(mito_dir, exist_ok=True)
    
    print("Downloading NCBI RefSeq mito sequences")
    
    base_url = "https://ftp.ncbi.nlm.nih.gov/refseq/release/mitochondrion/mitochondrion."
    suffix = ".genomic.fna.gz"
    files_to_get = ["1.1"] #
    
    all_files =[]
    downloaded_files = []
    
    for version in files_to_get:
        file_url = f"{base_url}{version}{suffix}"
        gz_filename = f"mito.{version}.genomic.fna.gz"
        fasta_filename = f"mito.{version}.genomic.fna"
        
        print(f"Downloading mito version {version}")
        
        # Download and extract the file
        try:
            extracted_path = fetch_and_extract(
                url=file_url,
                fetched_to=os.path.join(mito_dir, gz_filename),
                extract_to=mito_dir,
                expected_file=fasta_filename,
                logger=logger,
            )
            downloaded_files.append(extracted_path)
            all_files.append(extracted_path)
            all_files.append(os.path.join(mito_dir, gz_filename))
            print(f"Successfully downloaded and extracted {fasta_filename}")
        except Exception as e:
            logger.error(f"Failed to download mito version {version}: {e}")
            continue
    
    if not downloaded_files:
        logger.error("No mito files were successfully downloaded")
        return
    
    # Combine and deduplicate the sequences
    combined_fasta = os.path.join(mito_dir, "combined_mito_refseq.fasta")
    print(f"Combining and deduplicating {len(downloaded_files)} mito files")
    
    stats = remove_duplicates(
        input_file=downloaded_files,
        output_file=combined_fasta,
        by="seq",
        revcomp_as_distinct=False,  # Treat reverse complement as duplicate
        return_stats=True,
        logger=logger,
    )
    
    if stats:
        print(
            f"mito deduplication stats: {stats['unique_records']} unique sequences from {stats['total_records']} total, {stats['duplicates_removed']} duplicates removed"
        )
    
    # Clean up individual files to save space (optional)
    try:
        for file_path in all_files:
            if os.path.exists(file_path):
                os.remove(file_path)
                logger.debug(f"Removed intermediate file: {os.path.basename(file_path)}")
    except Exception as e:
        logger.warning(f"Could not remove intermediate mito files: {e}")
    
    print(f"mito sequences prepared in {mito_dir}")

temp_data_dir = "/clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/notebooks/Exprimental"
prepare_plastid_data(temp_data_dir, logger)
prepare_mito_data(temp_data_dir, logger)

## gene2accession (NCBI)

In [ ]:
# fetch_and_extract( url="http://ftp.ncbi.nlm.nih.gov/gene/DATA/gene2accession.gz",
#     fetched_to=os.path.join(rrna_dir, "gene2accession.gz"),
#     extract=False,
# )

silva_df = silva_df.with_columns(
    ncbi_taxonid=pl.col("ncbi_taxonid").cast(pl.String)
)

silva_df1 = silva_df.join(
    genbank_summary.select(["ncbi_taxonid", "ftp_path"]),
    on=["ncbi_taxonid"],
    how="left",
)
silva_df1

silva_df = silva_df.with_columns(
    genome_ftp_url=pl.when(pl.col("ncbi_taxonid").is_not_null())
    .then(
        pl.format(
            "https://ftp.ncbi.nlm.nih.gov/genomes/all/refseq/taxid_{}/",
            pl.col("ncbi_taxonid"),
        )
    )
    .otherwise(None),
    datasets_api_url=pl.when(pl.col("ncbi_taxonid").is_not_null())
    .then(
        pl.format(
            "https://api.ncbi.nlm.nih.gov/datasets/v2alpha/genome/taxon/{}/download?include_annotation_type=GENOME_FASTA,RNA_FASTA",
            pl.col("ncbi_taxonid"),
        )
    )
    .otherwise(None),
)

# Save metadata table
metadata_output = os.path.join(rrna_dir, "rrna_metadata.tsv")
silva_df.write_csv(metadata_output, separator="\t")
print(
    f"Saved rRNA metadata table with {len(silva_df)} entries to {metadata_output}"
)

gene2accession = pl.read_csv(
    os.path.join(rrna_dir, "gene2accession.gz"),
    separator="\t",
    # skip_rows=1,
    # infer_schema_length=100020,
    null_values=["na", "NA", "-"],
    ignore_errors=True,
    has_header=True,
    # n_rows=100
)
gene2accession.write_parquet(os.path.join(rrna_dir, "gene2accession.parquet"))
# gene2accession = pl.read_parquet(os.path.join(rrna_dir, "gene2accession.parquet"))
# gene2accession.collect_schema()
# Schema([('#tax_id', Int64),
#     ('GeneID', Int64),
#     ('status', String),
#     ('RNA_nucleotide_accession.version', String),
#     ('RNA_nucleotide_gi', String),
#     ('protein_accession.version', String),
#     ('protein_gi', Int64),
#     ('genomic_nucleotide_accession.version', String),
#     ('genomic_nucleotide_gi', Int64),
#     ('start_position_on_the_genomic_accession', Int64),
#     ('end_position_on_the_genomic_accession', Int64),
#     ('orientation', String),
#     ('assembly', String),
#     ('mature_peptide_accession.version', String),
#     ('mature_peptide_gi', String),
#     ('Symbol', String)])
gene2accession = gene2accession.rename({"#tax_id": "ncbi_taxonid"})
test_df = gene2accession.filter(pl.col("ncbi_taxonid").is_in(unique_taxids))
test_df.height # 148449745
test_df2 = gene2accession.select(["ncbi_taxonid","assembly"]).unique()
test_df2.height # 52548


